# Common REST API Demo

[Common REST API Docs](https://enclave-markets.notion.site/Common-REST-API-9d546fa6282b4bad87ef43d189b9071b)

## Getting started

Note that for many of the endpoints such as deposit and withdrawal to work, you cannot use the sandbox environment.

### Auth

Before using, please read the [overview docs](https://enclave-markets.notion.site/REST-API-Overview-57966a627e5445bba573fd66475a553d)
as well as the [authentication docs](https://enclave-markets.notion.site/Rest-API-Authentication-3956dcfecbdc48269031cf052926c90d) 
to get an API key with View + Transfer to use the full demo, or just make a key with View to ensure no funds are moved.

### Prepare account

After following the instruction to get an API key and secret, for all the endpoints to work as shown, do the following:

1.  provision a deposit address for AVAX
2.  deposit (not from a smart wallet!) some AVAX to the address
3.  add a withdrawal address to your address book

Now you're all set :)


## Setup

Install dependencies. 

### Janky fix import

And add the enclave module's path to the system path so we can import the local module in jupyter. 
Alternatively in VSCode you can follow [this tutorial](https://stackoverflow.com/a/73954768),
or from the command line generally [this tutorial](https://stackoverflow.com/a/65182353).

If you follow the above, feel free to remove the next cell.

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Dependencies

Continue installing and importing dependencies normally.

In [3]:
%pip install requests

import enclave

Note: you may need to restart the kernel to use updated packages.


## Environment variables

Set the following variables for auth and to set the base url.

In [4]:
import enclave.models
from IPython.core.error import UsageError

# change to the env of your choice that matches the API keys: PROD, DEV, SANDBOX etc.
BASE_URL = enclave.models.DEV


### three ways to get API key and secret, tried in order.

# 1. as hardcoded strings
API_KEY: str = ""
API_SECRET: str = ""

# 2. as environment variables
if not(all([API_KEY, API_SECRET])):
    try:
        API_KEY = %env ENCLAVE_API_KEY
        API_SECRET = %env ENCLAVE_API_SECRET
    except UsageError:
        pass

# 3. from a .env file with `key` and `secret` set.
if not(all([API_KEY, API_SECRET])):
    %pip install python-dotenv
    import dotenv
    envs = dotenv.dotenv_values("dev.env")
    if envs and "key" in envs and "secret" in envs:
        API_KEY, API_SECRET = str(envs["key"]), str(envs["secret"])


if not(all([API_KEY, API_SECRET])):
    raise ValueError("Please provide API_KEY and API_SECRET")

Note: you may need to restart the kernel to use updated packages.


## Create client

Create a client with the variables and get authed hello.

If you get anything other than a 200 status code and the hello message with `"success": True` 
and your Enclave Account ID, then there is an issue -- <br>
either with auth or connection permissions to the URL.

In [5]:
from enclave.client import Client

client = Client(api_key=API_KEY, api_secret=API_SECRET, base_url=BASE_URL)

hello_res = client.authed_hello()
if 200 <= hello_res.status_code < 300:
    print(hello_json := hello_res.json())
    assert hello_json["success"] == True
else:
    print(f"Could not connect:\n{hello_res.text=}")

{'success': True, 'result': '10458848535364347806'}


## Common API

Demonstrate common API endpoints.

### Market info

In [6]:
markets = client.get_markets().json()
print('available markets')
import json
print(json.dumps(markets["result"], indent=2))

available markets
{
  "cross": {
    "tradingPairs": [
      {
        "market": "AVAX-USDC",
        "pair": {
          "base": "AVAX",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0004
      },
      {
        "market": "AAVE-USDC",
        "pair": {
          "base": "AAVE",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
        "market": "BTC.b-USDC",
        "pair": {
          "base": "BTC.b",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
        "market": "ETH-USDC",
        "pair": {
          "base": "ETH",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
        "market": "LINK-USDC",
        "pair": {
          "base": "LINK",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
       

### Wallet

Demonstrate wallet endpoints such as getting balances, deposit addresses, and withdrawal addresses.

In [7]:
balances = client.get_balances().json()
print('available balances')
print(json.dumps(balances["result"], indent=2))

available balances
[
  {
    "coin": "AAVE",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "AVAX",
    "total": "0.02",
    "reserved": "0",
    "free": "0.02",
    "usdValue": "0.44"
  },
  {
    "coin": "BTC",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "BTC.b",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "ETH",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "LINK",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "USDC",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "USDCa",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "USDT",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "WBTC",
 

Per coin either manually or with get_balance

In [8]:
print(f'AVAX balance:\n{[x for x in balances["result"] if x["coin"] == "AVAX"]}')

balance_avax = client.get_balance("AVAX").json()
print(f'\n\nAVAX balance:\n{balance_avax["result"]=}')



AVAX balance:
[{'coin': 'AVAX', 'total': '0.02', 'reserved': '0', 'free': '0.02', 'usdValue': '0.44'}]


AVAX balance:
balance_avax["result"]={'accountId': '10458848535364347806', 'symbol': 'AVAX', 'totalBalance': '0.02', 'reservedBalance': '0', 'freeBalance': '0.02'}


Deposits and deposit addresses

In [9]:
deposit_addrs = client.get_deposit_addresses(["AVAX"]).json()
print(f"provisioned addresses:\n{json.dumps(deposit_addrs['result'], indent=2)}")

provisioned addresses:
[
  {
    "coin": "AVAX",
    "address": "b35521bf925d3c82f7417f2a90ed8209f2ecc94b"
  },
  {
    "coin": "AVAX",
    "address": "844978568f37b563b2f22f50737161dc822f80f4"
  },
  {
    "coin": "AVAX",
    "address": "f06f6d6fcf26b3bdd50fcd69e6070833463b2f11"
  },
  {
    "coin": "AVAX",
    "address": "3035fe787de6e3323387e0af31b0b7643d18e67f"
  },
  {
    "coin": "AVAX",
    "address": "0925f33294e4d4c5ee78d4b4e87286cbefc1bcac"
  },
  {
    "coin": "AVAX",
    "address": "c1b34bc32f79f24aec98432cf250fdca2beca95d"
  }
]


Provision a new address and see it come up

In [10]:
provision_res = client.provision_address("AVAX").json()
print(f"provisioned address:\n{json.dumps(provision_res['result'], indent=2)}")

new_deposit_addrs = client.get_deposit_addresses(["AVAX"]).json()

# there is one more address now
assert(len(new_deposit_addrs["result"]) > len(deposit_addrs["result"]))

provisioned address:
{
  "accountId": "10458848535364347806",
  "symbol": "AVAX",
  "address": "0x8972436683fd6b8e78544ffb9edce37f5edfb197"
}


In [11]:
%pip install pandas -q
import pandas as pd
from io import StringIO

deposits_csv = client.get_deposits_csv().text
print(f"raw csv: {deposits_csv=}")

x = pd.read_csv(StringIO(deposits_csv))
display(x)

Note: you may need to restart the kernel to use updated packages.
raw csv: deposits_csv='id,time,coin,size,status\n0xce6d811ce50db99f8681056a9d5c48dcd7f9b6532b77889b073f5610e64c7951,2023-11-30T19:08:01Z,AVAX,0.01,confirmed\n0x2d50ecc2dc1cb8b65a14b5fee392cbbe501fb4f875d3b20003e4a625e644c19d,2023-11-30T18:13:50Z,AVAX,0.01,confirmed\n'


,id,time,coin,size,status
0,0xce6d811ce50db99f8681056a9d5c48dcd7f9b6532b77...,2023-11-30T19:08:01Z,AVAX,0.01,confirmed
1,0x2d50ecc2dc1cb8b65a14b5fee392cbbe501fb4f875d3...,2023-11-30T18:13:50Z,AVAX,0.01,confirmed


Address book and withdrawals

Get one of the addresses and withdraw to it

In [12]:
# no current withdrawals
withdrawals = client.get_withdrawals().json()
print(f"withdrawals: {withdrawals=}")

withdrawals: withdrawals={'success': True, 'result': []}


In [13]:
addr_book = client.get_address_book().json()
print(f"address book: {addr_book}")

withdrawal_addr = addr_book["result"]["addressBook"][0]["withdrawalAddress"]
print(f"\n a {withdrawal_addr=}")

address book: {'success': True, 'result': {'addressBook': [{'withdrawalAddress': '0x15C23c69053b12Fe56F60e4c549d9aDc8d2d3bc9', 'label': 'Random account for demo', 'lastUpdated': '2023-11-30T21:49:12.544913759Z'}]}}

 a withdrawal_addr='0x15C23c69053b12Fe56F60e4c549d9aDc8d2d3bc9'


Withdraw to it

In [14]:
import decimal
import time
WITHDRAW_AMOUNT = decimal.Decimal(0.01)


withdrawal_res = client.withdraw(withdrawal_addr, WITHDRAW_AMOUNT, str(int(time.time())), "AVAX").json()
print(f"withdrawal: {withdrawal_res}")

withdrawal: {'success': True, 'result': {'withdrawal_status': 'WITHDRAWAL_PENDING', 'withdrawal_id': '57d3ff990a178a8911688a5291d1ac7bd22c46c732d066d383414c3b66f5d254', 'customer_withdrawal_id': '1701470141'}}


In [16]:
# there is a withdrawal now
withdrawals = client.get_withdrawals().json()
print(f"withdrawals: {withdrawals=}")

# get by txid
txid_withdrawal = withdrawals["result"][0]["txid"]
withdrawal_res = client.get_withdrawal(txid_withdrawal).json()
print(f"\nwithdrawal by txid: {withdrawal_res}")



withdrawals: withdrawals={'success': True, 'result': [{'coin': 'AVAX', 'size': '0.01000000000000000020816681711721685132943093776702880859375', 'status': 'WITHDRAWAL_SUCCESS', 'address': '0x15C23c69053b12Fe56F60e4c549d9aDc8d2d3bc9', 'withdrawal_id': '57d3ff990a178a8911688a5291d1ac7bd22c46c732d066d383414c3b66f5d254', 'txid': '0xa89e299d6ab0ec299c86711efb7975a8bc303f32005376135b7773bd7631197f', 'customer_withdrawal_id': '1701470141', 'time': '2023-12-01T22:35:41.982544061Z'}]}

withdrawal by txid: {'success': True, 'result': {'coin': 'AVAX', 'size': '0.01000000000000000020816681711721685132943093776702880859375', 'status': 'WITHDRAWAL_SUCCESS', 'address': '0x15C23c69053b12Fe56F60e4c549d9aDc8d2d3bc9', 'withdrawal_id': '57d3ff990a178a8911688a5291d1ac7bd22c46c732d066d383414c3b66f5d254', 'txid': '0xa89e299d6ab0ec299c86711efb7975a8bc303f32005376135b7773bd7631197f', 'customer_withdrawal_id': '1701470141', 'time': '2023-12-01T22:35:41.982544061Z'}}
